In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install timm
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import timm
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import seaborn as sn
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.8 MB/s eta 0:00:00


In [ ]:
import zipfile

# Replace 'dataset.zip' with the actual name of your uploaded ZIP file
zip_file_path = '/content/drive/MyDrive/Fenris/archive.zip'

# Replace '/content/dataset' with the desired path where you want to extract the contents
destination_folder = '/content/drive/MyDrive/Fenris'

# Unzip the dataset
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

print("Dataset has been successfully unzipped.")

Dataset has been successfully unzipped.


In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Set random seed for reproducibility
torch.manual_seed(42)

In [ ]:
# Define data transforms
data_transforms = transforms.Compose([
    transforms.Resize((224 , 224)),  # EfficientNet-B6 input size is (528, 528)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet mean and std
])

In [ ]:
# Set path to your train, val, and test directories
data_dir = '/content/drive/MyDrive/Fenris'
# Create train, val, and test datasets
train_dataset = ImageFolder(os.path.join(data_dir, '/content/drive/MyDrive/Fenris/train'), transform=data_transforms)
val_dataset = ImageFolder(os.path.join(data_dir, '/content/drive/MyDrive/Fenris/valid'), transform=data_transforms)
test_dataset = ImageFolder(os.path.join(data_dir, '/content/drive/MyDrive/Fenris/test'), transform=data_transforms)
# Create train, val, and test data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:
# Create model
num_classes = len(train_dataset.classes)
model = timm.create_model('efficientnet_b0', pretrained=True)

In [ ]:
#Training loop
num_epochs = 20
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move the model to the same device as the input data
model.to(device)

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    train_correct = 0
    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        train_correct += torch.sum(preds == labels.data)
    train_loss /= len(train_dataset)
    train_accuracy = train_correct.double() / len(train_dataset)
    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            val_correct += torch.sum(preds == labels.data)
    val_loss /= len(val_dataset)
    val_accuracy = val_correct.double() / len(val_dataset)
    checkpoint = {
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': train_loss,
        'train_accuracy': train_accuracy,
        'val_loss': val_loss,
        'val_accuracy': val_accuracy
    }
    torch.save(checkpoint, f'checkpoint_epoch_{epoch + 1}.pth')
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy.cpu().numpy())
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy.cpu().numpy())
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

  0%|          | 0/1323 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

Epoch [1/20], Train Loss: 0.9353, Train Accuracy: 0.7958, Val Loss: 0.2425, Val Accuracy: 0.9364


  0%|          | 0/1323 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

...

  0%|          | 0/42 [00:00<?, ?it/s]

Epoch [19/20], Train Loss: 0.0340, Train Accuracy: 0.9896, Val Loss: 0.1846, Val Accuracy: 0.9566


  0%|          | 0/1323 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

Epoch [20/20], Train Loss: 0.0372, Train Accuracy: 0.9889, Val Loss: 0.1849, Val Accuracy: 0.9581


In [ ]:
import matplotlib.pyplot as plt

# Plot Training and Validation Loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid()
plt.show()

# Plot Training and Validation Accuracy
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), train_accuracies, label='Training Accuracy')
plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid()
plt.show()

NameError: ignored

<Figure size 1000x600 with 0 Axes>